<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# LSEG Data Python Library

**Symbology**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers

* `RICs`
* Symbology conversions
* Working with `RICs`
* Working with data fields
* Working with Chain `RICs`

## Imports and Versions

The following imports several **packages** as used in the following.

In [4]:
import lseg.data as ld  # the LSEG Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [5]:
import sys
print(sys.version)

3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]


In [55]:
ld.__version__

1.5.1


In [7]:
np.__version__

'1.23.4'

In [8]:
pd.__version__

'1.5.3'

In [9]:
cf.__version__

'0.17.3'

## Connecting to LSEG's services

The LSEG Data Platform hosts many API Families which themselves host individual APIs which allow users to access data (e.g.: historical end-of-business-day (i.e.: close) prices for a company's publicly floated equity (i.e.: stock)) or services (e.g.: calculations). The [LSEG Data Python Library](https://developers.lseg.com/en) allows us to connect to this platform, and we can authenticate ourselves with the following code:

In [1]:
ld.open_session()

## RICs

A `RIC` is a **Reuters Instrument Code**.  It is used, amongst other things, to specify which instruments to retrieve historical data or, for example, news headlines. The following represents a list of `RICs` across several asset classes.

In [11]:
rics = [
    'GE',  # General Electric stock
    'AAPL.O',  # Apple stock
    '.SPX',  # S&P 500 stock index
    '.VIX',  # VIX volatility index
    'EUR=',  # EUR/USD exchange rate
    'XAU=',  # Gold price
    'DE10YT=RR',  # 10yr Bund price
]

`RICs` can be searched for in the **Data Item Browser**.

<img src="http://eikon.tpq.io/tr_eikon_dib_rics.png">

## Symbology Conversions

As there are a number of different symbology types (eg ISIN, SEDOL etc) used in financial markets &mdash; the ability to map between these different schemes is very useful. The function `ld.content.symbol_conversion` allows the conversion from one type of symbol to another one. There are several layers of service within the LD Lib.; you can find out more about them [here](https://developers.lseg.com/en); suffice to say that we are going to use the `content` layer when using the `symbol_conversion` function.

In [15]:
help(ld.content.symbol_conversion)

Help on package lseg.data.content.symbol_conversion in lseg.data.content:

NAME
    lseg.data.content.symbol_conversion

DESCRIPTION
    The SymbolConversion content object allows you to convert various types of symbols like RICs, ISINs, CUSIPs,
    and so on, using the Search/Lookup API of the Data Platform.

PACKAGE CONTENTS
    _asset_class
    _asset_state
    _country_code
    _definition
    _symbol_type

CLASSES
    enum.StrEnum(builtins.str, enum.ReprEnum)
        lseg.data.content.symbol_conversion._asset_class.AssetClass
        lseg.data.content.symbol_conversion._asset_state.AssetState
        lseg.data.content.symbol_conversion._country_code.CountryCode
        lseg.data.content.symbol_conversion._symbol_type.SymbolTypes
    lseg.data.content._content_provider_layer.ContentProviderLayer(lseg.data.content._content_provider_layer.ContentUsageLoggerMixin, lseg.data.delivery._data._data_provider_layer.DataProviderLayer)
        lseg.data.content.symbol_conversion._definition.D

For example, the `RICs` can be converted to `ISINs` (International Securities Identification Numbers).

In [20]:
ld.content.symbol_conversion.Definition(
    symbols=rics[:2],
    from_symbol_type=ld.content.symbol_conversion.SymbolTypes.RIC,
    to_symbol_types=[ld.content.symbol_conversion.SymbolTypes.ISIN]
    ).get_data().data.df

DocumentTitle     IssueISIN
GE      General Electric Co, Ordinary Share, NYSE Cons...  US3696043013
AAPL.O  Apple Inc, Ordinary Share, NASDAQ Global Selec...  US0378331005

Multilpe output types can also be specified.

In [22]:
ld.content.symbol_conversion.Definition(
    symbols=rics[:2],
    from_symbol_type=ld.content.symbol_conversion.SymbolTypes.RIC,
    to_symbol_types=[
        ld.content.symbol_conversion.SymbolTypes.ISIN,
        ld.content.symbol_conversion.SymbolTypes.TICKER_SYMBOL]
    ).get_data().data.df

DocumentTitle     IssueISIN  \
GE      General Electric Co, Ordinary Share, NYSE Cons...  US3696043013   
AAPL.O  Apple Inc, Ordinary Share, NASDAQ Global Selec...  US0378331005   

       TickerSymbol  
GE               GE  
AAPL.O         AAPL

There is no guarantee, however, that a conversion is possible.

In [23]:
ld.content.symbol_conversion.Definition(
    symbols=rics,
    from_symbol_type=ld.content.symbol_conversion.SymbolTypes.RIC,
    to_symbol_types=[
        ld.content.symbol_conversion.SymbolTypes.ISIN,
        ld.content.symbol_conversion.SymbolTypes.TICKER_SYMBOL]
    ).get_data().data.df

DocumentTitle     IssueISIN  \
GE         General Electric Co, Ordinary Share, NYSE Cons...  US3696043013   
AAPL.O     Apple Inc, Ordinary Share, NASDAQ Global Selec...  US0378331005   
.SPX                      S&P 500 Index - CBOE, Equity Index          <NA>   
.VIX              CBOE Market Volatility Index, Equity Index          <NA>   
EUR=       Euro/US Dollar FX Spot Rate, FX Spot Rate, REF...          <NA>   
XAU=       Gold Spot Multi-Contributor, Commodity Spot, C...          <NA>   
DE10YT=RR  Germany 10 Year Benchmark, Government Bond Ben...  DE000BU2Z023   

          TickerSymbol  
GE                  GE  
AAPL.O            AAPL  
.SPX              <NA>  
.VIX              <NA>  
EUR=              <NA>  
XAU=              <NA>  
DE10YT=RR         <NA>

Of course, other symbol types, such as `SEDOL` ([Stock Exchange Daily Official List](http://www.londonstockexchange.com/products-and-services/reference-data/sedol-master-file/documentation/sedol-technical-specification.pdf)) can also be converted **to** `RICs` &mdash; or `ISINs` to this end.

In [25]:
sedols = ['B1YW440', '0673123', 'B02J639', 'B1XZS82', '0045614', '0053673',
          '0989529', '0216238', '0969703', '0263494']

In [26]:
ld.content.symbol_conversion.Definition(
    symbols=sedols,
    from_symbol_type=ld.content.symbol_conversion.SymbolTypes.SEDOL,
    to_symbol_types=[
        ld.content.symbol_conversion.SymbolTypes.RIC,
        ld.content.symbol_conversion.SymbolTypes.ISIN]
    ).get_data().data.df

DocumentTitle     RIC  \
B1YW440  3i Group PLC, Ordinary Share, London Stock Exc...   III.L   
0673123  Associated British Foods PLC, Ordinary Share, ...   ABF.L   
B02J639  Admiral Group PLC, Ordinary Share, London Stoc...  ADML.L   
B1XZS82  Anglo American PLC, Ordinary Share, London Sto...   AAL.L   
0045614  Antofagasta PLC, Ordinary Share, London Stock ...  ANTO.L   
0053673  Ashtead Group PLC, Ordinary Share, London Stoc...   AHT.L   
0989529  AstraZeneca PLC, Ordinary Share, London Stock ...   AZN.L   
0969703  Babcock International Group PLC, Ordinary Shar...   BAB.L   
0263494  BAE Systems PLC, Ordinary Share, London Stock ...  BAES.L   

            IssueISIN  
B1YW440  GB00B1YW4409  
0673123  GB0006731235  
B02J639  GB00B02J6398  
B1XZS82  GB00B1XZS820  
0045614  GB0000456144  
0053673  GB0000536739  
0989529  GB0009895292  
0969703  GB0009697037  
0263494  GB0002634946

## Working with RICs

Historical data shall be retrieved for the following stocks.

In [27]:
symbols = ['US0378331005', 'US0231351067', 'US30303M1027', 'US4581401001']

The `ISINs` are easily converted to `RICs`.

In [28]:
rics = ld.content.symbol_conversion.Definition(
    symbols=sedols,
    from_symbol_type=ld.content.symbol_conversion.SymbolTypes.SEDOL,
    to_symbol_types=ld.content.symbol_conversion.SymbolTypes.RIC
    ).get_data().data.df

In [29]:
rics

DocumentTitle     RIC
B1YW440  3i Group PLC, Ordinary Share, London Stock Exc...   III.L
0673123  Associated British Foods PLC, Ordinary Share, ...   ABF.L
B02J639  Admiral Group PLC, Ordinary Share, London Stoc...  ADML.L
B1XZS82  Anglo American PLC, Ordinary Share, London Sto...   AAL.L
0045614  Antofagasta PLC, Ordinary Share, London Stock ...  ANTO.L
0053673  Ashtead Group PLC, Ordinary Share, London Stoc...   AHT.L
0989529  AstraZeneca PLC, Ordinary Share, London Stock ...   AZN.L
0969703  Babcock International Group PLC, Ordinary Shar...   BAB.L
0263494  BAE Systems PLC, Ordinary Share, London Stock ...  BAES.L

With the `RICs` available, one can then retrieve historical data.

In [30]:
rics = list(rics.RIC.values)
rics

['III.L',
 'ABF.L',
 'ADML.L',
 'AAL.L',
 'ANTO.L',
 'AHT.L',
 'AZN.L',
 'BAB.L',
 'BAES.L']

In [37]:
data = ld.get_history(
    rics,  # the RICs
    fields='TR.PriceClose',  # close field
    start='2024-01-01',  # start date
    end='2024-05-20')  # end date

In [38]:
data.tail()  # final five rows

Price Close  III.L  ABF.L  ADML.L   AAL.L  ANTO.L  AHT.L  AZN.L  BAB.L  BAES.L
Date                                                                          
2024-05-14    2950   2719  2696.0  2619.5  2280.0   5962  12246  519.5  1352.0
2024-05-15    2966   2711  2674.0  2625.0  2289.0   6086  12172  526.0  1363.0
2024-05-16    2977   2738  2671.0  2629.0  2284.0   6036  12166  531.0  1364.0
2024-05-17    2958   2730  2669.0  2677.5  2365.0   5764  12110  528.5  1361.5
2024-05-20    2977   2726  2698.0  2680.0  2393.0   5876  12096  538.5  1372.0

In [39]:
data.info()  # DataFrame meta information

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 97 entries, 2024-01-02 to 2024-05-20
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   III.L   97 non-null     Int64  
 1   ABF.L   97 non-null     Int64  
 2   ADML.L  97 non-null     Float64
 3   AAL.L   97 non-null     Float64
 4   ANTO.L  97 non-null     Float64
 5   AHT.L   97 non-null     Int64  
 6   AZN.L   97 non-null     Int64  
 7   BAB.L   97 non-null     Float64
 8   BAES.L  97 non-null     Float64
dtypes: Float64(5), Int64(4)
memory usage: 8.4 KB


The code that follows visualizes the **time series data** in the `CLOSE` column as a **line plot**.

In [40]:
data.normalize().iplot()

One can also retrieve **news headlines** for these `RICs`.

In [24]:
','.join(rics)

'AMZN.O,AAPL.O,FB.O,INTC.O'

In [42]:
ld.news.get_headlines(
    'R:{}'.format(','.join(rics)),  # the RICs
    start='2024-05-01T20:00:00',  # the starting time
    end='2024-05-27T22:00:00',  # the end time
    count=20)  # number of headlines

headline  \
versionCreated                                                               
2024-05-27 20:57:07.128  Inquest into the death of New Brunswick scrapy...   
2024-05-27 20:55:00.086  Datopotamab-Deruxtecan zeigt in Phase-III-Stud...   
2024-05-27 20:23:28.973  Inquest into the death of New Brunswick scrapy...   
2024-05-27 19:54:50.737  Babcock trades -0.36 percent lower on Monday 2...   
2024-05-27 19:29:48.445  (EN) ASTRAZENECA PLC ESG Web-based Reporting (...   
2024-05-27 19:27:16.742  German trading: Babcock International Group (B...   
2024-05-27 18:55:29.979  CONNECTICUT CHILDRENS MEDICAL CENTER - THE LAT...   
2024-05-27 18:49:59.032  CONNECTICUT CHILDRENS MEDICAL CENTER - THE LAT...   
2024-05-27 18:49:27.390  OMTHERA PHARMACEUTICALS, INC. - THE LATEST IND...   
2024-05-27 18:43:38.654  BAE SYSTEMS AB - THE LATEST INDUSTRY NEWS INCL...   
2024-05-27 18:42:47.346  BAE SYSTEMS AB - THE LATEST INDUSTRY NEWS INCL...   
2024-05-27 18:35:09.912  Egypt cooperates with AstraZeneca in heart and...   
2024-05-27 17:27:11.713  German trading: 3i Group (IGQ5: EUR35.40) clim...   
2024-05-27 17:27:09.456  German trading: BAE Systems (BSP: EUR16.74) hi...   
2024-05-27 17:27:09.233  German trading: Anglo American (NGLB: EUR30.65...   
2024-05-27 17:27:08.903  German trading: AstraZeneca (ZEG: EUR143.85) d...   
2024-05-27 17:27:08.706  German trading: Ashtead Group (0LC: EUR65.50) ...   
2024-05-27 17:27:06.989  German trading: AntofAGasta (FG1: EUR26.70) in...   
2024-05-27 17:18:32.224     AstraZeneca i el govern es renten les mans! --   
2024-05-27 17:14:00.098  Le datopotamab deruxtecan a montré une survie ...   

                                                              storyId  \
versionCreated                                                          
2024-05-27 20:57:07.128     urn:newsml:newsroom:20240527:nNRAssoh3s:0   
2024-05-27 20:55:00.086  urn:newsml:reuters.com:20240527:nBw2sL90fa:1   
2024-05-27 20:23:28.973        urn:link:webnews:20240527:nNRAsspll7:0   
2024-05-27 19:54:50.737     urn:newsml:newsroom:20240527:nNRAsspdlx:0   
2024-05-27 19:29:48.445  urn:newsml:reuters.com:20240527:nGLF1qH4l3:3   
2024-05-27 19:27:16.742     urn:newsml:newsroom:20240527:nNRAssob3f:0   
2024-05-27 18:55:29.979     urn:newsml:newsroom:20240527:nNRAssoznx:0   
2024-05-27 18:49:59.032     urn:newsml:newsroom:20240527:nNRAssnz4u:0   
2024-05-27 18:49:27.390     urn:newsml:newsroom:20240527:nNRAssozza:0   
2024-05-27 18:43:38.654     urn:newsml:newsroom:20240527:nNRAsso22q:0   
2024-05-27 18:42:47.346     urn:newsml:newsroom:20240527:nNRAssopvq:0   
2024-05-27 18:35:09.912        urn:link:webnews:20240527:nNRAssouqi:0   
2024-05-27 17:27:11.713     urn:newsml:newsroom:20240527:nNRAsso4bb:0   
2024-05-27 17:27:09.456     urn:newsml:newsroom:20240527:nNRAssp1lv:0   
2024-05-27 17:27:09.233     urn:newsml:newsroom:20240527:nNRAssoel7:0   
2024-05-27 17:27:08.903     urn:newsml:newsroom:20240527:nNRAssoel4:0   
2024-05-27 17:27:08.706     urn:newsml:newsroom:20240527:nNRAssncc1:0   
2024-05-27 17:27:06.989     urn:newsml:newsroom:20240527:nNRAssnjm5:0   
2024-05-27 17:18:32.224        urn:link:webnews:20240527:nNRAssobpx:0   
2024-05-27 17:14:00.098  urn:newsml:reuters.com:20240527:nBw6QD4Wya:1   

                        sourceCode  
versionCreated                      
2024-05-27 20:57:07.128  NS:CANPRE  
2024-05-27 20:55:00.086     NS:BSW  
2024-05-27 20:23:28.973  NS:CITYNE  
2024-05-27 19:54:50.737  NS:STOCKP  
2024-05-27 19:29:48.445  NS:GLFILE  
2024-05-27 19:27:16.742  NS:GERCOM  
2024-05-27 18:55:29.979  NS:NEWBIH  
2024-05-27 18:49:59.032  NS:NEWBIH  
2024-05-27 18:49:27.390  NS:NEWBIH  
2024-05-27 18:43:38.654  NS:NEWBIZ  
2024-05-27 18:42:47.346  NS:NEWBIZ  
2024-05-27 18:35:09.912  NS:WNCOMA  
2024-05-27 17:27:11.713  NS:GERCOM  
2024-05-27 17:27:09.456  NS:GERCOM  
2024-05-27 17:27:09.233  NS:GERCOM  
2024-05-27 17:27:08.903  NS:GERCOM  
2024-05-27 17:27:08.706  NS:GERCOM  
2024-05-27 17:27:06.989  NS:GERCOM  
20

## Working with Data Fields

The LSEG Data Python Library provides access to many different **data fields** for single or multiple instruments. Field names can be searched for in the [**Data Item Browser**](https://developers.lseg.com/en/video-catalog/data-item-browser). There are several ways of doing this &mdash; the filter button opens a high level menu if you want to see all fields in a particular category. If you know what you are looking for, say `Revenue` you can just type this in and fields will be suggested. If you have typed in an instrument before you will also see the values associated with the fields. This is very useful in finding the exact field you want.

<img src="https://developers.lseg.com/content/dam/devportal/articles/Summary-of-Common-LSEG-Refinitiv-APIs/e.png">

Consider the Apple, Inc. stock with `RIC` `AAPL.O`. For the this instrument, different fields of data shall be retrieved. The function to be used is `ld.get_data()`.

In [43]:
help(ld.get_data)

Help on function get_data in module lseg.data._access_layer.get_data_func:

get_data(universe: Union[str, Iterable[str]], fields: Union[str, Iterable[str], NoneType] = None, parameters: Union[str, dict, NoneType] = None, header_type: lseg.data.content._header_type.HeaderType = <HeaderType.TITLE: 2>) -> pandas.core.frame.DataFrame
    Retrieves pricing snapshots, as well as Fundamental and Reference data.
    
    Parameters
    ----------
    universe: str | list
        Instruments to request
    fields: str | list, optional
        Fields to request
    parameters: str | dict, optional
        Single key=value global parameter or dictionary of global parameters to request
    header_type: HeaderType, default HeaderType.TITLE
        If HeaderKey.TITLE - returns field title as column headers for data
        If HeaderKey.NAME - returns field name as column headers for data
        If HeaderKey.NAME_AND_TITLE - returns field name and title as column headers for data
    
    Returns
  

In [45]:
data = ld.get_data(
    'AAPL.O',
    ['TR.PriceClose', 'TR.Volume', 'TR.PriceLow',
     'TR.TotalReturnYTD', 'TR.TotalReturn52WK'])

In [46]:
data

Instrument  Price Close    Volume  Price Low  YTD Total Return  \
0     AAPL.O       189.98  36326975   188.0404          -1.06413   

   52 Week Total Return  
0              8.866185

The same methodology works for multiple `RICs`.

In [47]:
rics

['III.L',
 'ABF.L',
 'ADML.L',
 'AAL.L',
 'ANTO.L',
 'AHT.L',
 'AZN.L',
 'BAB.L',
 'BAES.L']

In [48]:
data = ld.get_data(rics, ['TR.PriceClose', 'TR.Volume', 'TR.PriceLow',
                                        'TR.TotalReturnYTD', 'TR.TotalReturn52WK'])

In [49]:
data

Instrument  Price Close   Volume  Price Low  YTD Total Return  \
0      III.L         2950   860368       2887         21.850475   
1      ABF.L         2722   671221       2624         14.997888   
2     ADML.L         2725   313114       2671          3.454739   
3      AAL.L         2615  5726567       2615         35.027757   
4     ANTO.L         2246   978649       2231          34.88308   
5      AHT.L         5714   446463       5662          4.878431   
6      AZN.L        12296   917582      12244         17.803109   
7      BAB.L          555   622075        540         40.506329   
8     BAES.L         1386  3996186       1378         26.602708   

   52 Week Total Return  
0              56.42891  
1             50.225003  
2             22.325114  
3             17.232152  
4             64.088656  
5             17.779943  
6              6.648451  
7             80.716482  
8             49.564591

In [50]:
cols = ['YTD Total Return', '52 Week Total Return']
data.set_index('Instrument')[cols].iplot(kind='bar')

## Working with Chain RICs

The `RIC` structure also provide access to groups of `RICs` that have a common association &mdash; for example, the constituent members of the FTSE 100 index or a list of options for a particular underlying contract. These groups are maintained by Thomson Reuters and are always up to date. It is important to note that chains contain a list of instruments only and not their values. 

For example:

* `0#.FTSE` gives you a list of FTSE 100 constituents 

* `0#GDAX*.EX` gives you a list of all options for all maturities for the DAX index
   
* `0#GDAXM8*.EX` gives you a list of all options for June 2018 maturity for the DAX index

Via Chain `RICs` it is convenient to retrieve data for e.g. for **all constituents of an index** at once.

In [52]:
dax = ld.get_data(
    universe='0#.GDAXI',
    fields=['TR.CommonName', 'TR.PriceClose','TR.Volume', 'TR.TotalReturnYTD'])

In [53]:
dax

Instrument                                Company Common Name  Price Close  \
0    BASFn.DE                                            BASF SE        48.76   
1    IFXGn.DE                           Infineon Technologies AG        38.06   
2    SIEGn.DE                                         Siemens AG       178.92   
3    EONGn.DE                                            E On Se        12.45   
4    DBKGn.DE                                   Deutsche Bank AG        15.71   
5   SATG_p.DE                                       Sartorius AG        253.6   
6     SAPG.DE                                             SAP SE       180.42   
7    BAYGn.DE                                           Bayer AG        28.27   
8     RWEG.DE                                             RWE AG        35.02   
9     1COV.DE                                        Covestro AG        49.71   
10  HNKG_p.DE                                Henkel AG & Co KGaA        83.66   
11    ZALG.DE                                         Zalando SE        24.05   
12   DTEGn.DE                                Deutsche Telekom AG        21.76   
13    MRCG.DE                                         Merck KGaA        167.4   
14  PSHG_p.DE                       Porsche Automobil Holding SE        49.92   
15  VOWG_p.DE                                      Volkswagen AG        120.4   
16    BMWG.DE                        Bayerische Motoren Werke AG         93.7   
17    BEIG.DE                                      Beiersdorf AG        145.9   
18    HEIG.DE                            Heidelberg Materials AG        99.82   
19    RHMG.DE                                     Rheinmetall AG        532.2   
20   MTXGn.DE                                MTU Aero Engines AG        234.1   
21    FREG.DE                             Fresenius SE & Co KGaA         29.2   
22    DHLn.DE                                   Deutsche Post AG        39.04   
23    ALVG.DE                                         Allianz SE        266.2   
24   MUVGn.DE  Muenchener Rueckversicherungs Gesellschaft in ...        462.5   
25    CONG.DE                                     Continental AG        62.12   
26    SHLG.DE                            Siemens Healthineers AG         53.8   
27    SY1G.DE                                         Symrise AG        106.1   
28   ADSGn.DE                                          Adidas AG        222.7   
29   HNRGn.DE                                  Hannover Rueck SE        226.8   
30   DTGGe.DE                           Daimler Truck Holding AG        39.44   
31   ENR1n.DE                                  Siemens Energy AG        25.82   
32    MBGn.DE                             Mercedes-Benz Group AG        66.29   
33   BNRGn.DE                                        Brenntag SE        65.66   
34    CBKG.DE                                     Commerzbank AG        15.59   
35    AIRG.DE                                          Airbus SE       159.42   
36    VNAn.DE                                         Vonovia SE        27.99   
37   DB1Gn.DE                                 Deutsche Boerse AG        182.9   
38     QIA.DE                                          Qiagen NV        40.25   
39  P911_p.DE                             Dr Ing hc F Porsche AG        75.84   

     Volume  YTD Total Return  
0   1081805           6.91979  
1   1566671            1.7485  
2    610332          8.286545  
3   2768856          6.745407  
4   3816885         30.719263  
5     77426        -23.731482  
6    490359         30.962007  
7   2309474           -15.603  
8   1491689        -12.332366  
9    825921         -5.637813  
10   140898         17.769985  
11   400362         12.121212  
12  3114117          3.678009  
13   117979         17.873712  
14   971322          7.772021  
15  1057607          7.692308  
16   507582          -1.24199  
17   116581          8.308308  
18   162940         27.101204  
19   115789         87.486341  
20    518

In [54]:
dax.set_index('Instrument')['YTD Total Return'].sort_values().iplot(kind='bar', color='blue')

## Conclusions

This tutorial covers:

* `RICs`(Reuters Instruments Codes)
* Symbology conversions
* Working with `RICs`
* Working with data fields
* Working with Chain `RICs`

The LSEG Data Python Library allows for **easy symbology conversions** and gives access to a **wealth of data fields** per instrument. The Data Item Browser allows to search for both `RICs` as well as data fields &mdash; and also provides values as well so you can see if it is the right field for you or not.

## LSEG Data Python Library Developer Resources

* [API Pages](https://developers.lseg.com/en/api-catalog) 
* [Q&A Forums](https://community.developers.refinitiv.com/index.html) 

Data Item Browser Application: Type `DIB` into Workspace Search Bar.

* [Article on Chains](https://developers.lseg.com/en/article-catalog/article/simple-chain-objects-ema-part-1)

<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">